In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [2]:
# CSV 파일 목록
csv_files = ['data/강원특별자치도_키워드.csv', 'data/경기도_키워드.csv', 'data/경상남도_키워드.csv', 'data/경상북도_키워드.csv', 
             'data/광주광역시_키워드.csv', 'data/대구광역시_키워드.csv', 'data/대전광역시_키워드.csv', 'data/부산광역시_키워드.csv', 
             'data/서울특별시_키워드.csv', 'data/세종특별자치시_키워드.csv', 'data/울산광역시_키워드.csv', 'data/인천광역시_키워드.csv', 
             'data/전라남도_키워드.csv', 'data/전라북도_키워드.csv', 'data/제주특별자치도_키워드.csv', 'data/충청남도_키워드.csv', 
             'data/충청북도_키워드.csv']

# 모든 CSV 파일을 리스트에 저장
all_dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    all_dfs.append(df)
    
# 모든 CSV 파일을 하나의 데이터프레임으로 합치기
all_df = pd.concat(all_dfs, ignore_index=True)

In [3]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70446 entries, 0 to 70445
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   시도       70446 non-null  object
 1   시군구      70446 non-null  object
 2   code     70446 non-null  int64 
 3   token    70446 non-null  object
 4   keyword  70446 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


In [4]:
grouped_all_df = all_df.groupby(['시도', '시군구', 'code'])['keyword'].apply(' '.join).reset_index()

In [5]:
grouped_all_df.head()

,시도,시군구,code,keyword
0,강원특별자치도,강릉시,42150,변동 강동면 미술관 바다 입장료 왕복 대관령 주차장 박물관 정동 오죽헌 심곡 정동진...
1,강원특별자치도,고성군,42820,적당하다 동해안 중요하다 아침 순서대로 먹다 유명하다 찜 무지개 소문나다 둘째 행복...
2,강원특별자치도,동해시,42170,용이하다 동해 여름 울퉁불퉁하다 추천 설치다 평평하다 장의 황금박쥐 신기하다 동굴 ...
3,강원특별자치도,삼척시,42230,동화책 완벽하다 알차다 전시관 건강하다 방문객 알록달록하다 온천 안전하다 초등학생 ...
4,강원특별자치도,속초시,42210,적당하다 동해안 중요하다 아침 순서대로 먹다 유명하다 찜 무지개 소문나다 둘째 행복...


In [6]:
grouped_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   시도       250 non-null    object
 1   시군구      250 non-null    object
 2   code     250 non-null    int64 
 3   keyword  250 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.9+ KB


In [7]:
tfidf_vectorizer = TfidfVectorizer()

# TF-IDF 행렬 계산
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped_all_df['keyword'])

In [8]:
tfidf_matrix

<250x56494 sparse matrix of type '<class 'numpy.float64'>'
	with 764337 stored elements in Compressed Sparse Row format>

In [9]:
# 키워드 추출 함수 정의
def tfidf_extract_keywords(tfidf_vectorizer, tfidf_matrix, top_n):
    # TF-IDF 행렬을 Numpy 배열로 변환
    tfidf_array = tfidf_matrix.toarray()
    feature_names = tfidf_vectorizer.get_feature_names_out()
    keywords = []
    for row in tfidf_array:
        # TF-IDF 값이 0이 아닌 인덱스 추출
        non_zero_idx = np.flatnonzero(row)  
        # top_n과 0이 아닌 인덱스 수 중 작은 값 선택
        n = min(top_n, non_zero_idx.size)
        # 상위 n개 키워드 인덱스 추출
        keywords_idx = non_zero_idx[np.argsort(row[non_zero_idx])[-n:]] 
        # 상위 n개 키워드 추출
        keywords.append(' '.join([feature_names[idx] for idx in keywords_idx]))
    return keywords

In [10]:
# tfidf_extract_keywords 함수 적용
top_keywords = tfidf_extract_keywords(tfidf_vectorizer, tfidf_matrix, top_n=3000)

In [11]:
grouped_all_df['keyword'] = top_keywords

In [12]:
grouped_all_df.head()

,시도,시군구,code,keyword
0,강원특별자치도,강릉시,42150,면사무소 로컬 미니 소장 희망이 런치 랜드마크 허브 지평선 연결하다 유형문화재 경쟁...
1,강원특별자치도,고성군,42820,대표 벚꽃 조명 마당 공연장 항상 모형 검색 특징 생태 촉촉하다 가성 사진촬영 힐링...
2,강원특별자치도,동해시,42170,당연하다 주소 전혀 전국 조선시대 건물 주변 운치 대표 벚꽃 자연환경 자리 생명 연...
3,강원특별자치도,삼척시,42230,여전하다 오늘 가을 성하다 역사 정갈하다 일상 대충 차다 검색 새롭다 내용 갈다 익...
4,강원특별자치도,속초시,42210,이루다 번의 몽환 화려하다 길어지다 용하다 싱겁다 커피한잔 평일 어린이집 전의 돈까...


In [14]:
grouped_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   시도       250 non-null    object
 1   시군구      250 non-null    object
 2   code     250 non-null    int64 
 3   keyword  250 non-null    object
dtypes: int64(1), object(3)
memory usage: 7.9+ KB


In [15]:
grouped_all_df.to_csv('data/전국_시군구_키워드.csv', encoding='utf-8-sig', index=False)